In [1]:
# --------------------------
# CONFIGURATION & IMPORTS
# --------------------------
import os
import time
import torch
import torchaudio
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from BEATs import BEATs, BEATsConfig

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT_PATH = "K:/DCASE/BEATs_iter3.pt"
TRAIN_DIR = "C:/DCASE_Temp/BEATs/Embeddings"
TEST_DIR = "K:/DCASE/BEATs/Test_Embeddings"
LABEL_DIR = "K:/DCASE/labels"
SAVE_DIR = "K:/DCASE"
MASK_PARAM = 80


In [2]:
# --------------------------
# LOAD BEATs MODEL
# --------------------------
print("\n🔄 Loading BEATs model...")
checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
cfg = BEATsConfig()
cfg.input_patch_size = (16, 16)
cfg.conv_bias = checkpoint["cfg"].get("conv_bias", False)
model = BEATs(cfg)
model.load_state_dict(checkpoint["model"], strict=False)
model.to(DEVICE)
model.eval()
print("✅ BEATs model ready!\n")

# --------------------------
# SpecAugment Function (Optional)
# --------------------------
def apply_specaugment(tensor, mask_param=MASK_PARAM):
    tensor = tensor.unsqueeze(0)
    tensor = torchaudio.transforms.FrequencyMasking(mask_param)(tensor)
    tensor = torchaudio.transforms.TimeMasking(mask_param)(tensor)
    return tensor.squeeze(0)

# --------------------------
# Embedding Extraction
# --------------------------
def extract_beats_embedding(wav_file, apply_aug=False):
    try:
        waveform, sr = torchaudio.load(wav_file)
    except Exception as e:
        print(f"[ERROR] Loading {wav_file}: {e}")
        return None

    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)

    waveform = waveform.to(DEVICE)
    waveform = torch.nn.functional.pad(waveform, (0, max(0, 16000*30 - waveform.shape[1])))
    waveform = waveform[:, :16000*30]

    with torch.no_grad():
        features = model.extract_features(waveform)[0].squeeze(0)
        if apply_aug:
            features = apply_specaugment(features)
        return features.cpu().numpy()

# --------------------------
# Process WAV files to .npy
# --------------------------
def process_wavs(input_dir, output_dir, apply_aug=False):
    os.makedirs(output_dir, exist_ok=True)
    for machine in os.listdir(input_dir):
        in_path = os.path.join(input_dir, machine)
        out_path = os.path.join(output_dir, machine)
        os.makedirs(out_path, exist_ok=True)

        for file in os.listdir(in_path):
            if file.endswith(".wav"):
                full_in = os.path.join(in_path, file)
                full_out = os.path.join(out_path, f"BEATs_{'aug_' if apply_aug else ''}{file.replace('.wav','.npy')}")

                if os.path.exists(full_out):
                    print(f"[SKIP] {file}")
                    continue

                emb = extract_beats_embedding(full_in, apply_aug)
                if emb is not None:
                    np.save(full_out, emb)
                    print(f"[SAVE] {full_out}")

# --------------------------
# Dataset
# --------------------------
class BEATsEmbeddingDataset(Dataset):
    def __init__(self, root_dir):
        self.paths = [
            os.path.join(dp, f) for dp, _, fs in os.walk(root_dir) for f in fs if f.endswith(".npy")
        ]

    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        data = np.load(self.paths[idx], mmap_mode='r').astype(np.float32)
        return torch.from_numpy(data), self.paths[idx]

class TestEmbeddingDataset(Dataset):
    def __init__(self, dir):
        self.paths = sorted([os.path.join(dir, f) for f in os.listdir(dir) if f.endswith(".npy")])

    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        x = np.load(self.paths[idx]).astype(np.float32)
        return torch.from_numpy(x), os.path.basename(self.paths[idx])


🔄 Loading BEATs model...


c:\Users\krishkannan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


✅ BEATs model ready!



In [9]:
# --------------------------
# Student Model
# --------------------------
class Bottleneck(nn.Module):
    def __init__(self, in_dim=768, out_dim=256):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim)
    def forward(self, x): return self.linear(x)

class StudentNet(nn.Module):
    def __init__(self, in_dim=256, out_dim=768):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_dim, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv1d(512, out_dim, 3, padding=1)
        )
    def forward(self, x): return self.net(x.transpose(1,2)).transpose(1,2)

# --------------------------
# Cosine Loss
# --------------------------
def cosine_loss(teacher, student):
    t = F.normalize(teacher, dim=-1)
    s = F.normalize(student, dim=-1)
    return 1 - (t * s).sum(dim=-1).mean()

In [10]:

# --------------------------
# Training Loop
# --------------------------
def train_model():
    dataset = BEATsEmbeddingDataset(TRAIN_DIR)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)

    model = nn.Sequential(Bottleneck(), StudentNet()).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    best_loss = float('inf')
    for epoch in range(20):
        model.train()
        total_loss = 0
        start = time.time()

        for x, _ in dataloader:
            x = x.to(DEVICE)
            bottleneck = model[0](x)
            output = model[1](bottleneck)
            loss = cosine_loss(x, output)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"[Epoch {epoch+1}] Loss: {avg_loss:.4f} | Time: {time.time()-start:.2f}s")

        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), os.path.join(SAVE_DIR, "student_model_best.pth"))

    torch.save(model.state_dict(), os.path.join(SAVE_DIR, "student_model_last.pth"))

In [30]:
# --------------------------
# Evaluation
# --------------------------
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset
class TestEmbeddingDataset(Dataset):
    def __init__(self, machine_dir):
        self.paths = sorted([
            os.path.join(machine_dir, f)
            for f in os.listdir(machine_dir)
            if f.endswith(".npy")
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        data = np.load(path).astype(np.float32)
        filename = os.path.splitext(os.path.basename(path))[0]
        return torch.from_numpy(data), filename

# Bottleneck Layer (reduce 768 → 256)
class Bottleneck(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(768, 256)

    def forward(self, x):
        return self.linear(x)

# StudentNet to match Conv1D structure in checkpoint
class StudentNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(512, 768, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = x.transpose(1, 2)      # (B, T, 256) → (B, 256, T)
        x = self.net(x)
        return x.transpose(1, 2)   # (B, 768, T)

# Evaluation
def evaluate_model_per_machine(model_checkpoint_path, test_dir, label_dir, output_csv_path="all_anomaly_scores.csv"):
    # Load model
    model = nn.Sequential(Bottleneck(), StudentNet()).to(device)
    checkpoint = torch.load(model_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint)
    model.eval()

    all_results = []

    for machine_type in sorted(os.listdir(test_dir)):
        machine_path = os.path.join(test_dir, machine_type)
        label_path = os.path.join(label_dir, f"{machine_type}.csv")

        if not os.path.isfile(label_path):
            print(f"[WARNING] Label file not found for {machine_type}. Skipping.")
            continue

        dataset = TestEmbeddingDataset(machine_path)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

        anomaly_scores = []

        with torch.no_grad():
            for x, filename in dataloader:
                x = x.to(device)  # (1, T, 768)
                student_input = model[0](x)
                output = model[1](student_input)

                # Cosine similarity
                teacher_norm = F.normalize(x, dim=-1)
                student_norm = F.normalize(output, dim=-1)
                cos_sim = (teacher_norm * student_norm).sum(dim=-1).mean().item()
                score = 1 - cos_sim  # Higher score = more anomalous

                anomaly_scores.append((filename[0], score))

        # Load labels
        df_labels = pd.read_csv(label_path)
        df_labels["filename"] = df_labels["filename"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

        df_scores = pd.DataFrame(anomaly_scores, columns=["filename", "score"])

        # Merge scores with labels
        df = df_scores.merge(df_labels, on="filename", how="inner")

        unmatched = len(df_scores) - len(df)
        if unmatched > 0:
            print(f"[DEBUG] {unmatched} unmatched filenames in {machine_type}: Sample -> {df_scores[~df_scores['filename'].isin(df['filename'])]['filename'].tolist()[:3]}")

        # Check label variety
        if df['label'].nunique() < 2:
            print(f"[{machine_type}] Skipped (insufficient label variety).")
            continue

        # Compute AUC and pAUC
        auc = roc_auc_score(df["label"], df["score"])
        p_auc = roc_auc_score(df["label"], df["score"], max_fpr=0.1)

        print(f"[{machine_type}] AUC: {auc:.4f} | pAUC: {p_auc:.4f}")

        df["machine_type"] = machine_type
        df["AUC"] = auc
        df["pAUC"] = p_auc
        all_results.append(df)

    if all_results:
        df_all = pd.concat(all_results, ignore_index=True)
        df_all.to_csv(output_csv_path, index=False)
        print(f"\n[INFO] All scores saved to: {output_csv_path}")
    else:
        print("[INFO] No valid evaluations were performed.")




In [ ]:
# --------------------------
# Entry Point
# --------------------------
if __name__ == "__main__":
    # To extract embeddings:
    # process_wavs("K:/DCASE/TestingData", TEST_DIR, apply_aug=False)
    # process_wavs("K:/DCASE/TrainingData", TRAIN_DIR, apply_aug=True)

    # To train the model:
    # train_model()

    # To evaluate:
    evaluate()

[test_bearing] Skipped.
[test_fan] Skipped.
[test_gearbox] Skipped.
[test_slider] Skipped.
[test_toycar] Skipped.
[test_toytrain] Skipped.
[test_valve] Skipped.

✅ All scores saved!


In [33]:
evaluate_model_per_machine(
    model_checkpoint_path="K:/DCASE/student_model_best.pth",
    test_dir="K:/DCASE/BEATs/Test_Embeddings",
    label_dir="K:/DCASE/generated_labels",
    output_csv_path="K:/DCASE/all_machine_scores.csv"
)

[test_bearing] AUC: 0.4938 | pAUC: 0.5095
[test_fan] AUC: 0.5215 | pAUC: 0.5016
[test_gearbox] AUC: 0.4716 | pAUC: 0.5037
[test_slider] AUC: 0.5768 | pAUC: 0.5079
[test_toycar] AUC: 0.5475 | pAUC: 0.5158
[test_toytrain] AUC: 0.6298 | pAUC: 0.5121
[test_valve] AUC: 0.5818 | pAUC: 0.5363

[INFO] All scores saved to: K:/DCASE/all_machine_scores.csv


In [32]:
import os
import pandas as pd

def generate_labels_from_filenames(test_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for machine_type in os.listdir(test_dir):
        machine_path = os.path.join(test_dir, machine_type)
        if not os.path.isdir(machine_path):
            continue

        data = []

        for file in os.listdir(machine_path):
            if file.endswith(".npy"):
                label = 1 if "anomaly" in file.lower() else 0
                filename = os.path.splitext(file)[0]  # No extension
                data.append((filename, label))

        df = pd.DataFrame(data, columns=["filename", "label"])
        output_csv_path = os.path.join(output_dir, f"{machine_type}.csv")
        df.to_csv(output_csv_path, index=False)
        print(f"[INFO] Saved: {output_csv_path} - {len(df)} samples")

# Example usage
generate_labels_from_filenames(
    test_dir="K:/DCASE/BEATs/Test_Embeddings",
    output_dir="K:/DCASE/generated_labels"
)

[INFO] Saved: K:/DCASE/generated_labels\test_bearing.csv - 200 samples
[INFO] Saved: K:/DCASE/generated_labels\test_fan.csv - 200 samples
[INFO] Saved: K:/DCASE/generated_labels\test_gearbox.csv - 200 samples
[INFO] Saved: K:/DCASE/generated_labels\test_slider.csv - 200 samples
[INFO] Saved: K:/DCASE/generated_labels\test_toycar.csv - 200 samples
[INFO] Saved: K:/DCASE/generated_labels\test_toytrain.csv - 200 samples
[INFO] Saved: K:/DCASE/generated_labels\test_valve.csv - 200 samples
